## Random forest classifeir with pipeline and hyperparmeter tuning


In [3]:
import seaborn as sns
df=sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [5]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [6]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [7]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

### we need to predict time

In [8]:
df.time

0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']

In [9]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [10]:
df.time

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [15]:
df['time'].value_counts()

time
0    176
1     68
Name: count, dtype: int64

In [11]:
df.time.unique()

array([0, 1])

In [12]:
## independent and dependent features
X=df.drop(labels=['time'],axis=1)
y=df.time

In [13]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [16]:
X['day'].value_counts()

day
Sat     87
Sun     76
Thur    62
Fri     19
Name: count, dtype: int64

In [14]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20, random_state=42)

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer #to handle missing values 
from sklearn.preprocessing import StandardScaler #feature scaling
from sklearn.preprocessing import OneHotEncoder #categorical to numerical
from sklearn.compose import ColumnTransformer

In [24]:
categorical_cols=df[['sex','smoker','day']]
numerical_cols=df[['total_bill','tip','size']]

In [23]:
categorical_cols

,sex,smoker,day
0,Female,No,Sun
1,Male,No,Sun
2,Male,No,Sun
3,Male,No,Sun
4,Female,No,Sun
...,...,...,...
239,Male,No,Sat
240,Female,Yes,Sat
241,Male,Yes,Sat
242,Male,No,Sat


In [27]:
## feature engineering automation
## Numerical Pipeling

num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),#missing values handle
        ('scalar',StandardScaler())#feature scaling
    ]
)

## Categorical pipeling

cat_pipeline=Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),#missing values
        ('Onehotencoder',OneHotEncoder()) #categorical features to numerical
    ]
)

In [29]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols.columns),
    ('cat_pipeline',cat_pipeline,categorical_cols.columns)
])

In [31]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [51]:
## Model training automation
models={
    'Random Forest':RandomForestClassifier(),
    'Logistic Regression':LogisticRegression(),
    'Decision Tree Classifier':DecisionTreeClassifier()
}

In [34]:
from sklearn.metrics import accuracy_score

In [35]:
def evaluate_model(X_train,X_test,y_train,y_test,models):

    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        #train model
        model.fit(X_train,y_train)

        #predict testing data
        y_test_pred=model.predict(X_test)

        #get accuracy scores for test data

        test_model_score=accuracy_score(y_test,y_test_pred)
        report[list(models.keys())[i]]=test_model_score

    return report

In [52]:
evaluate_model(X_train,X_test,y_train,y_test,models)

{'Random Forest': 0.9591836734693877,
 'Logistic Regression': 1.0,
 'Decision Tree Classifier': 0.9387755102040817}

In [53]:
classifier=RandomForestClassifier()

In [55]:
## hyperparameter tuning

params={'max_depth':[3,5,10,None],
        'n_estimators':[100,200,300],
        'criterion':['gini','entropy']
        }

In [56]:
from sklearn.model_selection import RandomizedSearchCV

In [57]:
cv=RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)

In [58]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.974 total time=   0.5s
[CV 2/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.923 total time=   0.6s
[CV 3/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.974 total time=   0.5s
[CV 4/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.923 total time=   0.5s
[CV 5/5] END criterion=gini, max_depth=5, n_estimators=300;, score=0.923 total time=   0.5s
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.923 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=100;, score=1.000 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.923 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.949 total time=   0.1s
[CV 

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [59]:
cv.best_params_

{'n_estimators': 100, 'max_depth': None, 'criterion': 'gini'}

In [60]:
classifier=RandomForestClassifier(n_estimators=100,max_depth=None,criterion='gini')

In [61]:
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [62]:
y_pred=classifier.predict(X_test)

In [63]:
y_pred

array([0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1])

In [64]:
print(accuracy_score(y_test,y_pred))

0.9591836734693877
